## Logistic回归模型

In [2]:
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#设定随机种子
torch.manual_seed(2017)

### 导入数据并可视化

In [9]:
f=open('./dataForLogisticRegression.txt','r')
data_list=[i.split('\n')[0].split(',') for i in f.readlines()]
data = [(float(i[0]), float(i[1]), float(i[2])) for i in data_list]
f.close()
# print(data)
# 标准化
x0_max=max([i[0] for i in data])
x1_max=max([i[1] for i in data])
data=[(i[0]/x0_max, i[1]/x1_max, i[2]) for i in data]
x0 = list(filter(lambda x:x[-1]==0.0, data)) #从list中筛选部分参数的写法!
x1 = list(filter(lambda x:x[-1]==1.0, data))